In [1]:
#Importamos la siguientes librerias

import numpy as np
import pandas as pd
%matplotlib inline
import sklearn
import sklearn.datasets
import sklearn.linear_model
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
#Cargamos nuestra base de datos 
data_set = pd.read_csv('tweets.csv')

In [9]:
#Verificamos las columnas vacias
data_set.isnull().sum()

tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                   5462
negativereason_confidence        4118
airline                             0
airline_sentiment_gold          14600
name                                0
negativereason_gold             14608
retweet_count                       0
text                                0
tweet_coord                     13621
tweet_created                       0
tweet_location                   4733
user_timezone                    4820
dtype: int64

In [10]:
#Eliminamos las columnas nulas
del data_set["airline_sentiment_gold"]
del data_set["negativereason_gold"]
del data_set["negativereason_confidence"]

In [11]:
#Seleccionamos los tweets positivosy negativos para nuestra TFIDF usaremos solo 10 tweets
tweets=data_set.loc[(data_set["airline_sentiment"]=="positive") | (data_set["airline_sentiment"]=="negative")]
textdata=tweets[["text","airline_sentiment"]].head(10)
textdata

,text,airline_sentiment
1,@VirginAmerica plus you've added commercials t...,positive
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative
5,@VirginAmerica seriously would pay $30 a fligh...,negative
6,"@VirginAmerica yes, nearly every time I fly VX...",positive
8,"@virginamerica Well, I didn't…but NOW I DO! :-D",positive
9,"@VirginAmerica it was amazing, and arrived an ...",positive
11,@VirginAmerica I &lt;3 pretty graphics. so muc...,positive
12,@VirginAmerica This is such a great deal! Alre...,positive
13,@VirginAmerica @virginmedia I'm flying your #f...,positive


In [12]:
#Eliminamos caracteres inecesarios
textdata=textdata['text'].str.replace("^@\\w+ *", "", case=True)

In [13]:
textdata

1     plus you've added commercials to the experienc...
3     it's really aggressive to blast obnoxious "ent...
4              and it's a really big bad thing about it
5     seriously would pay $30 a flight for seats tha...
6     yes, nearly every time I fly VX this “ear worm...
8                      Well, I didn't…but NOW I DO! :-D
9     it was amazing, and arrived an hour early. You...
11    I &lt;3 pretty graphics. so much better than m...
12    This is such a great deal! Already thinking ab...
13    @virginmedia I'm flying your #fabulous #Seduct...
Name: text, dtype: object

In [14]:
#Cada tweet o texto sera colocado en alldocuments 
alldocuments=[]
for document in textdata.values:
    alldocuments.append(document)

In [15]:
#Aplicamos una implementacion basica de tfidf para calcular que palabras son las mas importantes en los 10 tweets o documentos
#y lo calculamos mediante la funcion sklearn_tfidf.fit_transform
sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True)
sklearn_representation = sklearn_tfidf.fit_transform(alldocuments)

In [16]:
#Mostramos la informacion
sklearn_representation.data

array([ 0.36745042,  0.36745042,  0.24521722,  0.2132093 ,  0.36745042,
        0.36745042,  0.36745042,  0.29032986,  0.36745042,  0.27743999,
        0.27743999,  0.21921083,  0.27743999,  0.21921083,  0.27743999,
        0.27743999,  0.21921083,  0.27743999,  0.27743999,  0.27743999,
        0.27743999,  0.27743999,  0.18514896,  0.16098168,  0.16098168,
        0.30688755,  0.36334569,  0.36334569,  0.45986151,  0.36334569,
        0.30688755,  0.45178224,  0.24814937,  0.19606774,  0.24814937,
        0.24814937,  0.16560193,  0.19606774,  0.24814937,  0.24814937,
        0.24814937,  0.24814937,  0.24814937,  0.24814937,  0.24814937,
        0.24814937,  0.16560193,  0.19606774,  0.19606774,  0.19606774,
        0.16560193,  0.1439861 ,  0.16560193,  0.23747952,  0.3005614 ,
        0.3005614 ,  0.3005614 ,  0.3005614 ,  0.3005614 ,  0.3005614 ,
        0.3005614 ,  0.3005614 ,  0.3005614 ,  0.3005614 ,  0.20057898,
        0.46502628,  0.46502628,  0.46502628,  0.46502628,  0.36

In [17]:
#Para la clasificacion bayesiana me tome la libertad de utilizar la libreria nltk, puesto que tengo mas dominio utilizando
#esta libreria y la otra estoy en un proceso de aprendizaje aun
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk.metrics
#Las columnas seleccionadas son los tweets y el sentimiento que puede ser positivo o negativo
tweetsclass=tweets[["text","airline_sentiment"]] 
tweetsclass=tweetsclass.reset_index(drop=True) #Reseteamos los indices
#Separamos nuestros tweets de prueba y de entrenamiento
tweets2=tweetsclass[6000:8000]
tweets=tweetsclass.head(5000)

In [31]:
#Esta funcion me permite obtener todas las palabras de los tweets a analizar en una lista
def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

#Esta funcion agrupa las palabras con distancias mas cortas por su frecuencia
def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features


def extract_features(names):
    document_words = set(names)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [40]:
#Clasificacion bayesiana
tw=[]
for i in range(len(tweets["text"])):
    #tokenizer
    w=word_tokenize(tweets["text"][i])
    tw.append((w,tweets["airline_sentiment"][i]))

    
word_features = get_word_features(get_words_in_tweets(tw))

In [24]:
#obtenemos nuestro conjunto de datos para entrenar el modelo
training_set = nltk.classify.apply_features(extract_features, tw)

In [25]:
#Aplicamos el metodo de clasificacion bayesiana
classifier = nltk.NaiveBayesClassifier.train(training_set)
#Predecimos un tweet cualquiera
tweet = 'I had a good flight'
print (classifier.classify(extract_features(tweet.split())))

positive
